# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


## 0. Initiate {zero}

In [78]:
import pandas as pd
import numpy as np
import math
import json
from sklearn.preprocessing import MultiLabelBinarizer 
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [79]:
transcript = transcript.merge(transcript.value.apply(pd.Series),left_index=True,right_index=True)

In [80]:
transcript[transcript.event=='offer completed']

event                            person  time  \
12658   offer completed  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f     0   
12672   offer completed  fe97aa22dd3e48c8b143116a8403dd52     0   
12679   offer completed  629fc02d56414d91bca360decdfa9288     0   
12692   offer completed  676506bad68e4161b9bbaffeb039626b     0   
12697   offer completed  8f7dd3b2afe14c078eb4f6e6fe4ba97d     0   
12717   offer completed  227f2d69e46a4899b70d48182822cff6     0   
12721   offer completed  bb0f25e23a4c4de6a645527c275cd594     0   
12744   offer completed  d72d201be5794279aa716d8ad82b8d90     0   
12764   offer completed  73ffefd41e9a4ca3ab26b2b3697c6eb7     0   
12767   offer completed  3e621194f72e40d7a0b695ee9b7c38b7     0   
12780   offer completed  b860d355ef6e4c66b5d5a837c56ef32d     0   
12784   offer completed  99297ea01107436fa8c2e2bc86f55d89     0   
12786   offer completed  24115a61df25473e84a8a03f3c98de1a     0   
12798   offer completed  afce4cf8194f4e90a3e92da941a23601     0   
12817   offer completed  a2ffc22184d94d6fa21b924eb49c0d06     0   
12826   offer completed  2481f1fcfbcb4b288e5a03af02d95373     0   
12835   offer completed  c3321bc76743445c9103eef0ce9d833b     0   
12841   offer completed  162e7d9c644b438ea4606e337df0d3d1     0   
12862   offer completed  de90515e9f174ddfb8b4c677448286ef     0   
12865   offer completed  5152fa6375184287b06e2fd0d5abed34     0   
12900   offer completed  5d048a36203c483facb5ed09983ab388     0   
12936   offer completed  16dc4306258b4fef90ffe99f53ed7952     0   
12943   offer completed  2c9a9137efff4820a47d56dbf0fa555f     0   
12972   offer completed  bf7a630d566f46f79b855990daf2d0b0     0   
12988   offer completed  3b78a727167b432d8e1935227b286def     0   
12997   offer completed  6d11a027be6c4fac98d27eec516ad2f6     0   
13021   offer completed  0454292436bd4cee98e8ec873e30433e     0   
13036   offer completed  60b8a75638d2461b988ba4f089f8ad24     0   
13043   offer completed  9299c46942f84bb6b5e98fd78fbb9d41     0   
13056   offer completed  69d148898a80483babb0f9b2f2cccf5a     0   
...                 ...                               ...   ...   
306249  offer completed  2c2a1a03e7bf49febd39594ce9eb08bb   714   
306255  offer completed  3a17105cc12e4fc9aabc7c324835ba9c   714   
306260  offer completed  7718656997f3453db0f5aeca9cd35240   714   
306265  offer completed  2a109589d81b49fb891bd526e121c1c9   714   
306284  offer completed  0fa9cb0814914f778995e061d4460e1a   714   
306292  offer completed  8692060470dd472385a617029e19fc91   714   
306295  offer completed  264f1758fc9744a9badd3906364a598a   714   
306306  offer completed  0f1d40baf5fd438ea8d699a721467c1b   714   
306340  offer completed  845c31e647b449479f30b36eac5c5856   714   
306344  offer completed  b73c8d5fe3c84a79bb3432387f0f4fb5   714   
306348  offer completed  087d0fcdbf294f7494a96d8d6a56cf41   714   
306357  offer completed  a3fa9c5ae07348f0ad9c6942a17e063b   714   
306365  offer completed  557e2c8128334f00bbe020595885b9c4   714   
306374  offer completed  2d868dda028f445b8c7f592d33077229   714   
306376  offer completed  0dc85918181e4c3eb0e412d37fa8a552   714   
306404  offer completed  6f3fe50d1f26409f94489a85c766f79e   714   
306407  offer completed  7f0d9541dc284bb5b390c863644f04cd   714   
306410  offer completed  b13eaeb7a91e415a972c9ebc7f63ac57   714   
306423  offer completed  8771f96bc7f947c09a8f78ee2e14a1a7   714   
306430  offer completed  0355c6a5fdbc429ea5b05e8a9ecd2eae   714   
306440  offer completed  f74033a8c6834303bd1b3ef0c98d11d8   714   
306444  offer completed  84ad352984b04b73b6dbff8d719de321   714   
306455  offer completed  f3e801caeafe4899b3b989b586e74ac7   714   
306456  offer completed  f3e801caeafe4899b3b989b586e74ac7   714   
306470  offer completed  452cdae2c68e4732a4177d3929562690   714   
306475  offer completed  0c027f5f34dd4b9eba0a25785c611273   714   
306497  offer completed  a6f84f4e976f44508c358cc9aba6d2b3   714   
306506  offer completed  b895c57e8cd047a8872ce02aa54759d6   714  

In [81]:
transcript[transcript['event']=='offer completed'].value[12658]

{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}

## 1. Clean and Organize Data

In [82]:
def clean_and_organize_data(portfolio=portfolio,profile=profile,transcript=transcript):
    ## Portfolio
    portfolio.rename(columns={'id':'offer_id'},inplace=True) # match other table
    # one hot encode channels
    mlb = MultiLabelBinarizer() 
    portfolio = portfolio.join(pd.DataFrame(mlb.fit_transform(portfolio.pop('channels')),
                          columns=mlb.classes_,
                          index=portfolio.index))
    portfolio = portfolio.merge(pd.get_dummies(portfolio['offer_type'],prefix='offer',prefix_sep='_x_'),left_index=True,right_index=True)
    portfolio.drop(columns='offer_type',inplace=True)
    portfolio.set_index('offer_id')
    ## Profile
    profile = profile.dropna(axis = 0)
    profile.loc[:,'became_member_on']=pd.to_datetime(profile['became_member_on'],format='%Y%m%d')
    profile = profile.merge(pd.get_dummies(profile['gender'],prefix='gender',prefix_sep='_x_'),left_index=True,right_index=True)
    profile = profile.drop(columns='gender')
    #profile.set_index('id',inplace=True)
    ## Transcript
    # expand dict in value to its own columns per dict key
    transcript = transcript.merge(transcript.value.apply(pd.Series),left_index=True,right_index=True)
    # merge 
    transcript.loc[transcript['offer_id'].isna(),'offer_id'] = transcript[transcript['offer id'].isna()]['offer id']
    transcript = transcript.merge(pd.get_dummies(transcript['event'],prefix='event',prefix_sep='_x_'),left_index=True, right_index=True)
    transcript.drop(columns = ['offer id','value'],inplace=True)
    '''col_list = ['time','person',
                 'offer_id',
                 'amount',
                 'reward',
                 'event_x_offer received',
                 'event_x_offer viewed',
                 'event_x_transaction',
               'event_x_offer completed']'''
    #transcript = transcript[col_list]
    transcript = transcript[transcript['person'].isin(profile.index)].reset_index()
    t_offer = transcript[transcript['event']=='offer received']
    t_viewed = transcript[transcript['event']=='offer viewed']
    t_trans = transcript[transcript['event']=='transaction']
    t_compl = transcript[transcript['event']=='offer completed']
    
    return portfolio,profile,transcript,t_offer,t_viewed,t_trans,t_compl

In [83]:
portfolio_new,profile_new,transcript_new,t_offer,t_viewed,t_trans,t_compl = clean_and_organize_data()

C:\Users\thoma\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


KeyError: 'offer id'

In [ ]:
print(portfolio.shape)
print(profile.shape)
print(transcript.shape)

In [ ]:
print(portfolio_new.shape)
print(profile_new.shape)
print(transcript_new.shape)

In [ ]:
print(t_offer.shape)
print(t_viewed.shape)
print(t_compl.shape)
print(t_trans.shape)


In [ ]:
def create_matrix(df , cols = ['person','offer_id']):
    '''
    INPUT:
    df - pandas dataframe with person, offer_id columns
    
    OUTPUT:
    user_item - user offer matrix 
    
    Description:
    Return a matrix with user ids as rows and article ids on the columns with 1 values where a user interacted with 
    an article and a 0 otherwise
    '''
    matrix = df.pivot_table(index = cols[0] ,columns= cols[1] ,aggfunc=len ,fill_value=0)
    #user_item = df.groupby(['email','article_id'])[['title']].nunique().unstack()
    return matrix

In [ ]:
user_offer = create_matrix(t_offer, cols =['person','offer_id'])

In [ ]:
transcript_new

## 2. Explore Data

In [ ]:
tot_offers = t_offer.shape[0]
tot_viewed = t_viewed.shape[0]
tot_completed = t_compl.shape[0]
tot_num_purchased = t_trans.shape[0]
tot_sum_purch = t_trans.amount.sum()
tot_sum_reward = t_compl.reward.sum()

print('Percentage viewed : {0:.2f}%'.format(100* tot_viewed/tot_offers))
print('Percentage completed : {0:.2f}%'.format(100*tot_completed/tot_offers))
print('Percentage of purchases that were rewarded  : {0:.2f}%'.format(100*tot_completed/tot_num_purchased))

* we want to separate out transactions that were either not replaced to an offer nor not viewed.
* get transations that were not offers and 

In [ ]:
print(t_offer.shape)
t_offer.head()

In [ ]:
print(t_viewed.shape)
t_viewed.head()

In [ ]:
t_offer_view = t_offer.merge(t_viewed,how='outer',left_on=['person','offer_id'],right_on=['person','offer_id'])

In [ ]:
print(t_offer_view.shape)
t_offer_view.head()

In [ ]:
t_offer_view[t_offer_view['event_y'].isna()]

In [ ]:
offered.values

In [ ]:
offer_viewed = np.intersect1d(offered.values,viewed.values).tolist()

In [ ]:
transcript.loc[(transcript['offer_id'].isin(offer_viewed))&(transcript['person'].isin(test_1_ids)),'purc_viewed']=True

In [ ]:
transcript[transcript['person'].isin(test_1_ids)]

In [ ]:
def user_and_option(arr1, arr2):
    #function per user jterrace via Stack Overflow
    arr1_view = arr1.view([('',arr1.dtype)]*arr1.shape[1])
    arr2_view = arr2.view([('',arr2.dtype)]*arr2.shape[1])
    intersected = numpy.intersect1d(arr1_view, arr2_view)
    return intersected.view(arr1.dtype).reshape(-1, arr1.shape[1])